In [1]:
#!/usr/bin/env python
# coding: utf-8

# 필요 라이브러리 불러오기
import cv2
import numpy as np
import time
import serial
import serial.tools.list_ports
import imutils
from imutils.perspective import four_point_transform
from tensorflow.keras.models import model_from_json

ModuleNotFoundError: No module named 'imutils'

In [2]:
# USB 시리얼 포트 찾기

ports = serial.tools.list_ports.comports()
com = ''
for port, desc, hwid in sorted(ports):
    if 'USB' in desc or hwid:
        com = port
if com != '':
    print('\n microbit USB detected: ', com)
else:
    print('\n Please connect your microbit to this PC via USB')
    exit()


 microbit USB detected:  COM4


In [3]:
# 시리얼 통신으로 컴퓨터에 연결되어 있는 마이크로비트에 명령 보내기

def SerialSendCommand(ser, cmd):   
    cmd = str(cmd) 
    cmd  = cmd + '\n'
    cmd = str.encode(cmd) 
    ser.write(cmd)   

In [4]:
# Brain AI Car Body 마이크로비트로부터 전송받은 명령 확인하기

def SerialReceiveResponse(ser):
    response = 1
    line = ser.readline()
    line = str(line)
    line = ''.join([i for i in line if i.isdigit()])
    
    if line:

        if line == "1": 
            response = -1
        else:
            response = 0            

    return response

In [5]:
# 시리얼 통신을 통한 명령 전송 확인

def SerialConfirm(ser, cmd, tic, timeout):
    ret = 1
    
    if (time.time() - tic) > timeout: 
        SerialSendCommand(ser, cmd)
        tic = time.time()
        
    response = SerialReceiveResponse(ser)
    
    if response == 0:
        ret = 0 

    elif response == -1:
        ret = -1
    return ret, tic

In [6]:
def LineTracing(frame, ser, tic, cropFrame, width, height, y1, turn):
    frameArea = frame.shape[0] * frame.shape[1]
    
    #remove green color
    hsv = cv2.cvtColor(cropFrame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0, 80, 28), (75, 255,255))
    mask2 = cv2.inRange(hsv, (170, 80, 28), (180, 255,255))
    mask = mask + mask2
    cv2.imshow("mask2", mask)
    noGreen = cropFrame.copy()
    # replace with white
    noGreen[mask > 0] = (255, 255, 255)
    cv2.imshow("noGreen", noGreen)

    #find black line
    gray = cv2.cvtColor(noGreen, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY_INV)
    cv2.imshow("thresh", thresh)
    
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    largestArea = 0
    largestRect = None
    
    if len(cnts) > 0:
    
        c = max(cnts, key=cv2.contourArea)
        M = cv2.moments(c)
        
        for cnt in cnts:
 
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            sideOne = np.linalg.norm(box[0]-box[1])
            sideTwo = np.linalg.norm(box[0]-box[3])
            
            area = sideOne * sideTwo
            
            if area > largestArea:
                largestArea = area
                largestRect = box

    if largestArea > frameArea * 0.01:
        
        
        if M['m00'] == 0:
            pass

        else:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            
            cv2.line(frame,(cx,0), (cx, int(height)), (255, 0 , 0), 1)
            cv2.line(frame,(0,cy + y1), (int(width), cy + y1), (255, 0, 0), 1)
            cv2.drawContours(frame, cnts, -1, (0, 0 , 255), 1, offset = (0,y1))
                        
            turn = cx / width*100
            turn = turn - 50
            turn = turn * sens
            turn = turn - align
            turn = abs(int(turn))
            maxTurn = 55
            
            if turn < (align - maxTurn):
                turn = (align - maxTurn)
                SerialSendCommand(ser, turn)
                tic = time.time()
                text = str(turn)
            
            elif turn > (align + maxTurn):
                turn = (align + maxTurn)
                SerialSendCommand(ser, turn)
                tic = time.time()
                text = str(turn)

            else:
                SerialSendCommand(ser, turn)
                tic = time.time()
                text = str(turn)

    else:
        if time.time() - tic > 5:
            SerialSendCommand(ser, 0)
            print("I don't see the line")
            tic = time.time()
  
    return tic, turn

In [7]:
# 횡단보도에 도달함

def RedDetection(cropFrame, frame, y1):

    ret = 0
    frameArea = frame.shape[0] * frame.shape[1]

    hsv = cv2.cvtColor(cropFrame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0, 50, 50), (10, 255,255))
    mask2 = cv2.inRange(hsv, (170, 50, 50), (179, 255,255))
    mask = mask + mask2
    
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    #cv2.imshow("red detection", mask)

    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

    largestArea = 0
    largestRect = None

    if len(cnts) > 0:
        
 
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            sideOne = np.linalg.norm(box[0]-box[1])
            sideTwo = np.linalg.norm(box[0]-box[3])
            
            area = sideOne * sideTwo
            
            if area > largestArea:
                largestArea = area
                largestRect = box

    if largestArea > frameArea * 0.05:
     
        for i in range(0, 4):
            largestRect[i] += [0, y1]
        cv2.drawContours(frame, [largestRect], 0, (0,0,255) , 2)
        ret = 1        

    return ret, frame

In [8]:
# 신호 찾기

def FindTrafficSign(grabbed, frame):

    lower_hsv = np.array([85,100,70])
    upper_hsv = np.array([115,255,255])

    if not grabbed:
        return

    frameArea = frame.shape[0] * frame.shape[1]

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_hsv, upper_hsv)  

    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
   
    largestArea = 0
    largestRect = None

    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    
    if len(cnts) > 0:
       
        for cnt in cnts:
            
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            sideOne = np.linalg.norm(box[0]-box[1])
            sideTwo = np.linalg.norm(box[0]-box[3])
            area = sideOne * sideTwo
            
            if area > largestArea:
                
                largestArea = area
                largestRect = box

    if largestArea > frameArea*0.1:
        
        cv2.drawContours(frame, [largestRect], 0, (0,0,255), 2)

    cropped = None
    
    if largestRect is not None:
        
        cropped = four_point_transform(frame, [largestRect][0])

    return frame, cropped, largestRect

In [9]:
# 신호 읽기

def ReadTrafficSign(cropped, model):
    
    ret = -1

    if cropped is None:
        return ret

    input_width = 48
    input_height = 48
    
    resized_image = cv2.resize(cropped, (input_width, input_height)).astype(np.float32)
    
    normalized_image = resized_image / 255.0

    batch = normalized_image.reshape(1, input_height, input_width, 3)
    result_onehot = model.predict(batch)
    
    left_score, forward_score, stop_score, schoolZone_score, noSign_score = result_onehot[0]
    class_id = np.argmax(result_onehot, axis=1)[0] 

    if class_id == 0:
        detectedSign = 'Left'
        score = left_score
        ret = 1
        
    elif class_id == 1:
        detectedSign = 'Forward'
        score = forward_score
        ret = 1
        
    elif class_id == 2:
        detectedSign = 'Stop'
        score = stop_score 
        ret = 1
        
    elif class_id == 3:
        detectedSign = 'School Zone'
        score = schoolZone_score 
        ret = 1
        
    elif class_id == 4:
        detectedSign = 'No Sign'
        score = noSign_score 
        ret = 1
        
    return ret, detectedSign, score

In [10]:
# 신호에 따라 주행하기

def Move(ser, turn, turnTime, tic):
    ret = -1
    SerialSendCommand(ser, turn)
    if time.time() - tic > turnTime:
        ret = 1
    return ret

In [11]:
# 메인 함수

def Main():
    
    MODE_PAUSE = -1 
    MODE_LINE_TRACING = 0
    MODE_CROSSWALK = 1
    MODE_INFERENCE = 2
    MODE_MOVE = 3
    MODE_SCHOOLZONE = 4
    MODE_STOP = 5
    MODE_FORWARD = 6
    
    turn = align
    mode_status = MODE_PAUSE
    printTrafficSign = None 
    
    ser = serial.Serial(com, 115200, timeout=0, parity=serial.PARITY_NONE, rtscts=0)      
    
    with open('./model/model.json', 'r') as file_model:        
        model_desc = file_model.read()
        model = model_from_json(model_desc)
    model.load_weights('./model/weights.h5')

    tic = time.time()
    video_capture = cv2.VideoCapture(video)
    grabbed, frame = video_capture.read()
    width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH )
    height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT )
    
    
    print("Press 'q' to quit") 
    print("Press 'p' to start/pause")
    
    
    while(True):
        largestRect = None
        line = ser.readline()
        grabbed, frame = video_capture.read()

        y1 = int(height - int(height * .2))
        y2 = int(height - int(height * .0))
        x1 = 0
        x2 = int(width)
        cropFrame = frame[y1:y2, x1:x2]  
        
        if mode_status == MODE_LINE_TRACING:
            
            ret, frame,  = RedDetection(cropFrame, frame, y1)
            
            if ret == 1:
                mode_status = MODE_CROSSWALK
            else:
                tic, turn = LineTracing(frame, ser, tic, cropFrame, width, height, y1, turn)

        elif mode_status == MODE_CROSSWALK:
            SerialSendCommand(ser,speed)  
            
            cmd = 0
            ret, tic = SerialConfirm(ser, cmd, tic, .1)
            
            if ret == 0:
                mode_status = MODE_INFERENCE
                
        elif mode_status == MODE_INFERENCE:
            printTrafficSign = " "
            frame, cropped, largestRect = FindTrafficSign(grabbed, frame)
            
            if cropped is not None :
                ret, detectedTrafficSign, score = ReadTrafficSign(cropped, model)
                printTrafficSign = detectedTrafficSign + ' ' + str('%.0f' % (score*100)) + '%' 
                if ret == 1:
                        if score > 0.95 and "No Sign" not in detectedTrafficSign and time.time() - tic > 2:
                            tic = time.time()
                            if 'Left' in detectedTrafficSign:
                                mode_status = MODE_MOVE
                                SerialSendCommand(ser, turnSpeed)  
                                endTurn = align + turnDegree    
                                
                            elif 'Forward' in detectedTrafficSign:
                                mode_status = MODE_FORWARD
                                SerialSendCommand(ser, turnSpeed)  
                                
                            elif 'School Zone' in detectedTrafficSign:
                                mode_status = MODE_SCHOOLZONE
                                cmd = align
                            elif 'Stop' in detectedTrafficSign:
                                mode_status = MODE_STOP
                                cmd = '-1'
                                
        elif mode_status == MODE_MOVE:
            SerialSendCommand(ser, turnSpeed)  
            #slowly increase the the turn angle
            cmd = int(align + turnDegree/(turnTime / (time.time() - tic)))
            ret = Move(ser, cmd, turnTime, tic)

            if ret == 1: 
        
                SerialSendCommand(ser, speed) 
                mode_status = MODE_LINE_TRACING
                tic = time.time()
                     
        
        elif mode_status == MODE_FORWARD:
                if turn == align: #to prevent division by zero
                    turn += .05
                
                cmd = int(align + (turn - align)/(forwardTime / (forwardTime - (time.time() - tic))))
                
                ret = Move(ser, cmd, forwardTime, tic)
                
                if ret == 1: 
                
                    SerialSendCommand(ser, speed)  
                    mode_status = MODE_LINE_TRACING
                    tic = time.time()
                    

        elif mode_status == MODE_SCHOOLZONE:
            cmd = schoolSpeed
            SerialSendCommand(ser,cmd)
            
            y1 = int(height - int(height * .3))
            cropFrame = frame[y1:y2, x1:x2]
            
            tic2 = LineTracing(frame, ser, tic, cropFrame, width, height, y1, turn)
            
            if time.time() - tic > 4:
                mode_status = MODE_LINE_TRACING
                
        elif mode_status == MODE_STOP:

            cmd = "-1"
            SerialSendCommand(ser, cmd)            
            break     
            
        key = cv2.waitKey(1);
        
        if key == ord('q'):
                break
       
        elif key == ord('p'):
            if mode_status != MODE_PAUSE:
                mode_status = MODE_PAUSE
                cmd = '0'      
                SerialSendCommand(ser, cmd)                
                time.sleep(.25)
                print("Mode: Paused")
                SerialSendCommand(ser, cmd)
            
            elif mode_status == MODE_PAUSE:
                mode_status = MODE_LINE_TRACING
                SerialSendCommand(ser, speed)
                time.sleep(.25)
                print ("Mode: Line Tracing")
                SerialSendCommand(ser, speed)
       
        if largestRect is not None and printTrafficSign is not None: 
                cv2.drawContours(frame, [largestRect], 0, (0,0,255) ,2)
                cv2.putText(frame, printTrafficSign, tuple(largestRect[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (37, 222, 0), 3)
     
        cv2.imshow('Brain AI Car', frame)
    
    cmd = '0' 
    SerialSendCommand(ser, cmd)
    time.sleep(.5)
    cmd = '0'  
    SerialSendCommand(ser, cmd)
            
    print("Closing Serial")
    ser.close()
    video_capture.release()
    cv2.destroyAllWindows()
    print('Shutting Down')

In [12]:
video = 'http://192.168.137.32:8080/video' #'http://192.168.0.14:8080/video'
threshold = 100 # put to 175 if using at night with a LED light
speed = 600         # speed during line tracing
turnSpeed = 500     # speed when turning
schoolSpeed = 400   # speed in the school zone
forwardTime = 1.5      # how long the car will go forward
turnTime = 3.5    # how long the car will turn
turnDegree = 60     # maximum turn degree while turning
align = 87
sens = 1.8
 
if __name__ == '__main__': 
    Main()    

FileNotFoundError: [Errno 2] No such file or directory: './model/model.json'